```
pip install --pre pystan
```

In [1]:
import nest_asyncio; nest_asyncio.apply()
import stan

import numpy as np
import numpyro; numpyro.enable_x64()
from numpyro.infer import MCMC, NUTS
from age_model import get_data, transform_data, model

In [2]:
data = get_data()
transformed_data = transform_data(data)
kernel = NUTS(model, step_size=0.02, max_tree_depth=15, target_accept_prob=0.95)
mcmc = MCMC(kernel, num_warmup=0, num_samples=1)
mcmc.run(np.zeros(2, dtype=np.uint32), transformed_data)
z = {k: v[0] for k, v in mcmc.get_samples().items()}

sd_dip_rnde <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
phi <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_log_ifr_age_rnde_mid1 <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_log_ifr_age_rnde_mid2 <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_log_ifr_age_rnde_old <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
log_relsusceptibility_age_reduced <numpyro.distributions.constraints._Real object at 0x7f898de976d0> (2,)
sd_upswing_timeeff_reduced <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_timeeff_shift_mid1 <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
impact_intv_children_effect <numpyro.distributions.constraints._Interval object at 0x7f89747c4dc0> ()
impact_intv_onlychildren_effect <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975

  0%|          | 0/1 [00:00<?, ?it/s]

sd_dip_rnde <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
phi <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_log_ifr_age_rnde_mid1 <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_log_ifr_age_rnde_mid2 <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_log_ifr_age_rnde_old <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
log_relsusceptibility_age_reduced <numpyro.distributions.constraints._Real object at 0x7f898de976d0> (2,)
sd_upswing_timeeff_reduced <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
hyper_timeeff_shift_mid1 <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975b0> ()
impact_intv_children_effect <numpyro.distributions.constraints._Interval object at 0x7f8974561760> ()
impact_intv_onlychildren_effect <numpyro.distributions.constraints._GreaterThan object at 0x7f898de975

sample: 100%|██████████| 1/1 [00:09<00:00,  9.33s/it, 63 steps of size 2.00e-02. acc. prob=1.00]


In [6]:
with open("covid19AgeModel_v120_cmdstanv.stan") as f:
    stan_code = f.read()
stan_model = stan.build(stan_code, data=data, random_seed=1)
stan_fit = stan_model.sample(num_chains=1, num_warmup=20, stepsize=0.000001, max_depth=5, num_samples=2)

Building...
Messages from stanc:
  The parameter R0 was declared but was not used in the density calculation.
  The parameter beta was declared but was not used in the density
  calculation.
  The parameter dip_rnde was declared but was not used in the density
  calculation.
  The parameter hyper_log_ifr_age_rnde_mid1 was declared but was not used in
  the density calculation.
  The parameter hyper_log_ifr_age_rnde_mid2 was declared but was not used in
  the density calculation.
  The parameter hyper_log_ifr_age_rnde_old was declared but was not used in
  the density calculation.
  The parameter hyper_timeeff_shift_mid1 was declared but was not used in the
  density calculation.
  The parameter impact_intv_children_effect was declared but was not used in
  the density calculation.
  The parameter impact_intv_onlychildren_effect was declared but was not used
  in the density calculation.
  The parameter log_ifr_age_base was declared but was not used in the density
  calculation.
  The p

In [7]:
params = stan_fit.to_frame().to_dict()
z0 = {}
z1 = {}
for k in stan_fit.param_names:
    if k not in z:
        continue
    z0[k] = np.array([v[0] for k1, v in params.items() if k1.startswith(k)]).reshape(z[k].shape)
    z1[k] = np.array([v[1] for k1, v in params.items() if k1.startswith(k)]).reshape(z[k].shape)
pe0 = params["lp__"][0]
pe1 = params["lp__"][1]
print("stan pe diff:", pe1 - pe0)
potential_fn = mcmc.sampler._potential_fn_gen(transformed_data)
print("numpyro pe diff:", potential_fn(z1) - potential_fn(z0))

stan pe diff: -0.07387661714378879
numpyro pe diff: 1381.658727045331


In [8]:
params

{'lp__': {0: 24.77826944521704, 1: 24.70439282807325},
 'accept_stat__': {0: 1.0, 1: 0.9999999161499432},
 'stepsize__': {0: 0.0023518109108320432, 1: 0.0023518109108320432},
 'treedepth__': {0: 5.0, 1: 5.0},
 'n_leapfrog__': {0: 31.0, 1: 31.0},
 'divergent__': {0: 0.0, 1: 0.0},
 'energy__': {0: 50.77834309365302, 1: 22.34028703853374},
 'R0.1': {0: 0.11534586532820995, 1: 0.11357498285403257},
 'R0.2': {0: 1.6908750428557202, 1: 1.6612706628573044},
 'R0.3': {0: 13.785203819089881, 1: 13.37317429380662},
 'R0.4': {0: 9.39638925702539, 1: 9.189583457934479},
 'e_cases_N0.1': {0: 64.7281548557109, 1: 67.03700456770439},
 'e_cases_N0.2': {0: 71.11606952361694, 1: 70.99420429389177},
 'e_cases_N0.3': {0: 56.76349991312398, 1: 56.577741980514965},
 'e_cases_N0.4': {0: 117.22986386823696, 1: 118.09615197179136},
 'beta.1': {0: 1.5759057155681147, 1: 1.5969887576150046},
 'beta.2': {0: 1.2468859949292128, 1: 1.2410152037812956},
 'dip_rnde.1': {0: -0.6641668873366032, 1: -0.6650896448134654}